In [ ]:
# Run this code to analyse the whole database of videos and process it

In [1]:
# import os
# import base64
# import imageio
# import logging
# from dotenv import load_dotenv
# from scenedetect import VideoManager, SceneManager
# from scenedetect.detectors import ContentDetector
# from concurrent.futures import ThreadPoolExecutor

# # Configure logging
# logging.basicConfig(level=logging.INFO)

# # Load OpenAI API key from .env file
# load_dotenv()
# api_key = os.getenv("OPENAI_API_KEY")

# def analyze_video(video_path, threshold=27.0):
#     if not os.path.exists(video_path):
#         raise FileNotFoundError(f"The video file {video_path} does not exist.")
    
#     video_manager = VideoManager([video_path])
#     scene_manager = SceneManager()
#     scene_manager.add_detector(ContentDetector(threshold=threshold))

#     video_manager.set_downscale_factor()
#     video_manager.start()

#     scene_manager.detect_scenes(frame_source=video_manager)
#     scene_list = scene_manager.get_scene_list()

#     video_manager.release()

#     logging.info(f'Detected {len(scene_list)} scenes:')
#     for i, scene in enumerate(scene_list):
#         logging.info(f'Scene {i + 1}: Start {scene[0].get_timecode()} / Frame {scene[0].get_frames()}, '
#               f'End {scene[1].get_timecode()} / Frame {scene[1].get_frames()}')

#     return scene_list

# def process_frame(reader, frame_idx, output_path):
#     try:
#         frame = reader.get_data(frame_idx)
#         imageio.imwrite(output_path, frame)
#         with open(output_path, "rb") as image_file:
#             return base64.b64encode(image_file.read()).decode('utf-8')
#     except Exception as e:
#         logging.error(f"Failed to extract frame {frame_idx}: {e}")
#         return None

# def extract_frames_imageio(video_path, scenes, output_dir):
#     os.makedirs(output_dir, exist_ok=True)
#     reader = imageio.get_reader(video_path, 'ffmpeg')
#     key_frames_base64 = []

#     with ThreadPoolExecutor() as executor:
#         for i, scene in enumerate(scenes):
#             start_frame, end_frame = scene[0].get_frames(), scene[1].get_frames()
#             key_frame_indices = [start_frame, (start_frame + end_frame) // 2, end_frame]

#             futures = [
#                 executor.submit(process_frame, reader, frame_idx, os.path.join(output_dir, f'scene_{i + 1}_frame_{idx + 1}_{frame_idx}.jpg'))
#                 for idx, frame_idx in enumerate(key_frame_indices)
#             ]

#             for future in futures:
#                 result = future.result()
#                 if result:
#                     key_frames_base64.append(result)

#     return key_frames_base64

# def process_videos_in_directory(directory_path, output_base_dir, threshold=27.0):
#     # Get all video files in the directory
#     video_files = [f for f in os.listdir(directory_path) if f.endswith(('.mp4', '.avi', '.mkv'))]

#     if not video_files:
#         logging.info("No video files found in the directory.")
#         return

#     for video_file in video_files:
#         video_path = os.path.join(directory_path, video_file)
#         video_name = os.path.splitext(video_file)[0]
        
#         # Create a folder named after the video inside the output_base_dir
#         video_output_dir = os.path.join(output_base_dir, video_name)
#         scenes_output_dir = os.path.join(video_output_dir, 'scenes_output')

#         os.makedirs(scenes_output_dir, exist_ok=True)

#         logging.info(f"Processing video: {video_file}")
        
#         # Analyze the video
#         scenes = analyze_video(video_path, threshold)

#         # Extract key frames using imageio and encode in Base64
#         key_frames_base64 = extract_frames_imageio(video_path, scenes, scenes_output_dir)

#         logging.info(f"Finished processing video: {video_file}")
#         logging.info(f"Extracted {len(key_frames_base64)} key frames.")


# # Main script execution
# video_directory = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/02_Video_DB'
# output_base_directory = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB'

# # Process all videos in the directory
# process_videos_in_directory(video_directory, output_base_directory)


INFO:root:Processing video: Bananas_in_pyjamas.mp4
ERROR:pyscenedetect:VideoManager is deprecated and will be removed.
INFO:pyscenedetect:Loaded 1 video, framerate: 25.000 FPS, resolution: 640 x 360
INFO:pyscenedetect:Downscale factor set to 2, effective resolution: 320 x 180
INFO:pyscenedetect:Detecting scenes...
INFO:root:Detected 159 scenes:
INFO:root:Scene 1: Start 00:00:00.000 / Frame 0, End 00:00:08.360 / Frame 209
INFO:root:Scene 2: Start 00:00:08.360 / Frame 209, End 00:00:19.320 / Frame 483
INFO:root:Scene 3: Start 00:00:19.320 / Frame 483, End 00:00:24.040 / Frame 601
INFO:root:Scene 4: Start 00:00:24.040 / Frame 601, End 00:00:24.800 / Frame 620
INFO:root:Scene 5: Start 00:00:24.800 / Frame 620, End 00:00:26.600 / Frame 665
INFO:root:Scene 6: Start 00:00:26.600 / Frame 665, End 00:00:28.600 / Frame 715
INFO:root:Scene 7: Start 00:00:28.600 / Frame 715, End 00:00:29.960 / Frame 749
INFO:root:Scene 8: Start 00:00:29.960 / Frame 749, End 00:00:31.880 / Frame 797
INFO:root:Scene

In [2]:
import os
import base64
import aiohttp
import asyncio
import json
from PIL import Image
import numpy as np
import colorsys
import re
import aiofiles
import nest_asyncio
import imageio
import logging
from dotenv import load_dotenv
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector
from concurrent.futures import ThreadPoolExecutor

# Apply nest_asyncio to handle the running event loop in Jupyter
nest_asyncio.apply()

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load OpenAI API key from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Function to encode the image asynchronously
async def encode_image(image_path):
    async with aiofiles.open(image_path, "rb") as image_file:
        content = await image_file.read()
        return base64.b64encode(content).decode('utf-8')

# Function to determine the specific primary or special color category
def get_color_category(color):
    r, g, b = [x / 255.0 for x in color]
    h, l, s = colorsys.rgb_to_hls(r, g, b)

    primary_hues = {
        "red": (0.0, 0.1),  
        "yellow": (0.1, 0.18),
        "green": (0.25, 0.4),
        "blue": (0.55, 0.75),
    }

    for color_name, hue_range in primary_hues.items():
        if hue_range[0] <= h <= hue_range[1]:
            return color_name

    if (l >= 0.9 and s <= 0.1):
        return "white"
    if (l <= 0.1 and s <= 0.1):
        return "black"

    return "non-primary"

# Function to analyze image colors
def analyze_image_colors(image_path):
    image = Image.open(image_path)
    image = image.convert('RGB')
    data = np.array(image)

    unique_colors, counts = np.unique(data.reshape(-1, data.shape[2]), axis=0, return_counts=True)
    total_pixels = int(counts.sum())

    color_counts = {
        "Red": 0,
        "Yellow": 0,
        "Green": 0,
        "Blue": 0,
        "White": 0,
        "Black": 0,
        "Non-primary": 0
    }

    for color, count in zip(unique_colors, counts):
        category = get_color_category(tuple(color))
        color_counts[category.capitalize()] += int(count)

    color_percentages = {color: (count / total_pixels) * 100 for color, count in color_counts.items()}
    primary_total = color_counts["Red"] + color_counts["Yellow"] + color_counts["Blue"]
    color_dominance = "Primary colors" if primary_total > color_counts["Non-primary"] else "Non-primary colors"

    color_analysis_result = {
        "Color Analysis": {
            "Colors Found": {
                color: {
                    "Pixel Count": count,
                    "Percentage": f"{color_percentages[color]:.2f}%"
                } for color, count in color_counts.items()
            },
            "Dominance": color_dominance
        }
    }
    
    return color_analysis_result

# Async function to send the image to OpenAI API
async def send_image_to_openai(image_path, base64_image):
    api_key = os.getenv("OPENAI_API_KEY")
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": """
                        Analyze the following image and provide a detailed description in the format of JSON only. Ensure the output is strictly in JSON format without any additional text or code block formatting...
                        """
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 750
    }

    async with aiohttp.ClientSession() as session:
        async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload) as response:
            if response.status != 200:
                print(f"Request failed with status code: {response.status}")
                return None
            
            content = await response.json()
            content_text = content.get('choices', [{}])[0].get('message', {}).get('content', '').strip()

            try:
                content_dict = json.loads(content_text)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                return None

            return content_dict

# Async function to process all scenes in the folder
async def process_scenes_output(output_dir, json_output_dir):
    os.makedirs(json_output_dir, exist_ok=True)

    scenes = sorted([f for f in os.listdir(output_dir) if f.endswith('.jpg')], key=extract_scene_number)

    tasks = []
    for i, scene in enumerate(scenes):
        tasks.append(process_single_scene(i, scene, output_dir, json_output_dir))

    await asyncio.gather(*tasks)

# Async function to process a single scene
async def process_single_scene(i, scene, output_dir, json_output_dir):
    scene_path = os.path.join(output_dir, scene)
    base64_image = await encode_image(scene_path)
    color_analysis_result = analyze_image_colors(scene_path)
    openai_response = await send_image_to_openai(scene_path, base64_image)

    if not openai_response:
        print(f"Skipping {scene} due to invalid OpenAI response.")
        return

    final_output = {
        "Image File": scene,
        "Image Analysis": {
            **color_analysis_result["Color Analysis"],
            **openai_response.get("Image Analysis", {})
        }
    }

    output_filename = f'scene_{i + 1}_analysis.json'
    output_path = os.path.join(json_output_dir, output_filename)

    async with aiofiles.open(output_path, 'w') as json_file:
        await json_file.write(json.dumps(final_output, indent=4))

    print(f"Analysis for {scene} saved as {output_filename}")

# Helper function to extract scene numbers from filenames
def extract_scene_number(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else -1

# Original video processing functions
def analyze_video(video_path, threshold=27.0):
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"The video file {video_path} does not exist.")
    
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector(threshold=threshold))

    video_manager.set_downscale_factor()
    video_manager.start()

    scene_manager.detect_scenes(frame_source=video_manager)
    scene_list = scene_manager.get_scene_list()

    video_manager.release()

    logging.info(f'Detected {len(scene_list)} scenes:')
    for i, scene in enumerate(scene_list):
        logging.info(f'Scene {i + 1}: Start {scene[0].get_timecode()} / Frame {scene[0].get_frames()}, '
              f'End {scene[1].get_timecode()} / Frame {scene[1].get_frames()}')

    return scene_list

def process_frame(reader, frame_idx, output_path):
    try:
        frame = reader.get_data(frame_idx)
        imageio.imwrite(output_path, frame)
        with open(output_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        logging.error(f"Failed to extract frame {frame_idx}: {e}")
        return None

def extract_frames_imageio(video_path, scenes, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    reader = imageio.get_reader(video_path, 'ffmpeg')
    key_frames_base64 = []

    with ThreadPoolExecutor() as executor:
        for i, scene in enumerate(scenes):
            start_frame, end_frame = scene[0].get_frames(), scene[1].get_frames()
            key_frame_indices = [start_frame, (start_frame + end_frame) // 2, end_frame]

            futures = [
                executor.submit(process_frame, reader, frame_idx, os.path.join(output_dir, f'scene_{i + 1}_frame_{idx + 1}_{frame_idx}.jpg'))
                for idx, frame_idx in enumerate(key_frame_indices)
            ]

            for future in futures:
                result = future.result()
                if result:
                    key_frames_base64.append(result)

    return key_frames_base64

# Main function to process videos and analyze images
def process_videos_in_directory(directory_path, output_base_dir, threshold=27.0):
    # Get all video files in the directory
    video_files = [f for f in os.listdir(directory_path) if f.endswith(('.mp4', '.avi', '.mkv'))]

    if not video_files:
        logging.info("No video files found in the directory.")
        return

    for video_file in video_files:
        video_path = os.path.join(directory_path, video_file)
        video_name = os.path.splitext(video_file)[0]
        
        # Create a folder named after the video inside the output_base_dir
        video_output_dir = os.path.join(output_base_dir, video_name)
        scenes_output_dir = os.path.join(video_output_dir, 'scenes_output')
        json_output_dir = os.path.join(video_output_dir, 'json_output')

        os.makedirs(scenes_output_dir, exist_ok=True)

        logging.info(f"Processing video: {video_file}")
        
        # Analyze the video and extract scenes
        scenes = analyze_video(video_path, threshold)
        key_frames_base64 = extract_frames_imageio(video_path, scenes, scenes_output_dir)

        # Analyze all scenes and save the JSON output
        loop = asyncio.get_event_loop()
        if loop.is_running():
            task = loop.create_task(process_scenes_output(scenes_output_dir, json_output_dir))
        else:
            loop.run_until_complete(process_scenes_output(scenes_output_dir, json_output_dir))

        logging.info(f"Finished processing video: {video_file}")
        logging.info(f"Extracted {len(key_frames_base64)} key frames.")

# Main script execution
video_directory = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/02_Video_DB'
output_base_directory = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB'

# Process all videos in the directory
process_videos_in_directory(video_directory, output_base_directory)


INFO:root:Processing video: Bananas_in_pyjamas.mp4
ERROR:pyscenedetect:VideoManager is deprecated and will be removed.
INFO:pyscenedetect:Loaded 1 video, framerate: 25.000 FPS, resolution: 640 x 360
INFO:pyscenedetect:Downscale factor set to 2, effective resolution: 320 x 180
INFO:pyscenedetect:Detecting scenes...
INFO:root:Detected 159 scenes:
INFO:root:Scene 1: Start 00:00:00.000 / Frame 0, End 00:00:08.360 / Frame 209
INFO:root:Scene 2: Start 00:00:08.360 / Frame 209, End 00:00:19.320 / Frame 483
INFO:root:Scene 3: Start 00:00:19.320 / Frame 483, End 00:00:24.040 / Frame 601
INFO:root:Scene 4: Start 00:00:24.040 / Frame 601, End 00:00:24.800 / Frame 620
INFO:root:Scene 5: Start 00:00:24.800 / Frame 620, End 00:00:26.600 / Frame 665
INFO:root:Scene 6: Start 00:00:26.600 / Frame 665, End 00:00:28.600 / Frame 715
INFO:root:Scene 7: Start 00:00:28.600 / Frame 715, End 00:00:29.960 / Frame 749
INFO:root:Scene 8: Start 00:00:29.960 / Frame 749, End 00:00:31.880 / Frame 797
INFO:root:Scene

Request failed with status code: 429
Skipping scene_101_frame_1_10467.jpg due to invalid OpenAI response.
Request failed with status code: 429
Skipping scene_49_frame_1_4822.jpg due to invalid OpenAI response.
Request failed with status code: 429
Skipping scene_142_frame_1_15131.jpg due to invalid OpenAI response.
Request failed with status code: 429
Request failed with status code: 429
Request failed with status code: 429
Skipping scene_119_frame_1_12915.jpg due to invalid OpenAI response.
Skipping scene_90_frame_1_9263.jpg due to invalid OpenAI response.
Skipping scene_24_frame_1_2497.jpg due to invalid OpenAI response.
Request failed with status code: 429
Request failed with status code: 429
Skipping scene_77_frame_1_7515.jpg due to invalid OpenAI response.
Skipping scene_50_frame_1_4866.jpg due to invalid OpenAI response.
Request failed with status code: 429
Request failed with status code: 429
Skipping scene_60_frame_1_5802.jpg due to invalid OpenAI response.
Skipping scene_63_fra

In [3]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import base64
import aiohttp
import asyncio
import json
from PIL import Image
import numpy as np
import colorsys
import aiofiles
import nest_asyncio
from tqdm.asyncio import tqdm

# Apply nest_asyncio to handle the running event loop
nest_asyncio.apply()

# Function to encode the image asynchronously
async def encode_image(image_path):
    async with aiofiles.open(image_path, "rb") as image_file:
        content = await image_file.read()
        return base64.b64encode(content).decode('utf-8')

# Function to determine the specific primary or special color category
def get_color_category(color):
    r, g, b = [x / 255.0 for x in color]
    h, l, s = colorsys.rgb_to_hls(r, g, b)

    primary_hues = {
        "red": (0.0, 0.1),  
        "yellow": (0.1, 0.18),
        "green": (0.25, 0.4),
        "blue": (0.55, 0.75),
    }

    for color_name, hue_range in primary_hues.items():
        if hue_range[0] <= h <= hue_range[1]:
            return color_name

    if (l >= 0.9 and s <= 0.1):
        return "white"
    if (l <= 0.1 and s <= 0.1):
        return "black"

    return "non-primary"

# Function to analyze image colors and return as a dictionary
def analyze_image_colors(image_path):
    image = Image.open(image_path)
    image = image.convert('RGB')
    data = np.array(image)

    unique_colors, counts = np.unique(data.reshape(-1, data.shape[2]), axis=0, return_counts=True)
    total_pixels = int(counts.sum())

    color_counts = {
        "Red": 0,
        "Yellow": 0,
        "Green": 0,
        "Blue": 0,
        "White": 0,
        "Black": 0,
        "Non-primary": 0
    }

    for color, count in zip(unique_colors, counts):
        category = get_color_category(tuple(color))
        color_counts[category.capitalize()] += int(count)

    color_percentages = {color: (count / total_pixels) * 100 for color, count in color_counts.items()}
    primary_total = color_counts["Red"] + color_counts["Yellow"] + color_counts["Blue"]
    color_dominance = "Primary colors" if primary_total > color_counts["Non-primary"] else "Non-primary colors"

    color_analysis_result = {
        "Color Analysis": {
            "Colors Found": {
                color: {
                    "Pixel Count": count,
                    "Percentage": f"{color_percentages[color]:.2f}%"
                } for color, count in color_counts.items()
            },
            "Dominance": color_dominance
        }
    }
    
    return color_analysis_result

# Asynchronous function to send the image to OpenAI API
async def send_image_to_openai(image_path, base64_image):
    api_key = os.getenv("OPENAI_API_KEY")
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": """
                        Analyze the following image and provide a detailed description in the format of JSON only. Ensure the output is strictly in JSON format without any additional text or code block formatting. The JSON should include the following standardized labels:

                        1. **Image Analysis**: The root dictionary containing all analysis data.
                        
                        2. **Suitability**:
                            - "Nudity": Boolean indicating the presence of nudity.
                            - "Obscene Gestures": Boolean indicating the presence of obscene gestures.
                            - "Alcohol": Boolean indicating the presence of alcohol.
                            - "Drugs": Boolean indicating the presence of drugs.
                            - "Addictions": Boolean indicating the presence of addictions.

                        3. **Objects**:
                            - "Total Objects Identified": Integer representing the total number of objects identified.
                            - "Average Features Per Object": Float representing the average number of features per object.
                            - "Objects Details": Dictionary containing details of each object, where each object is labeled as "Object_1", "Object_2", etc., with the following structure:
                                - "Name": The name of the object.
                                - "Color": The color of the object.
                                - "Features": List of features of the object.
                                - "Total Features": Integer representing the number of features for the object.

                        4. **Place**:
                            - "Certainty": String indicating the certainty level of the identified place.
                            - "Fantasy/Adventurous Place": Boolean (0-1) indicating whether the place is classified as a fantasy/adventurous place or not.
                            - "Explanation": Detailed explanation of why the place is classified as fantasy/adventurous or not. Fantasy places are those that do not exist in reality, and adventurous places are defined as those involving clear statements of traveling to space or another country.

                        5. **Characters**:
                        - "Total Characters Identified": Integer representing the total number of characters identified.
                        - "Average Features Per Character": Float representing the average number of features per character.
                        - "Character Details": Dictionary containing details of each character, where each character is labeled as "Character_1", "Character_2", etc., with the following structure:
                            - "Name": The name of the character, if identifiable, or "Unknown" otherwise.
                            - "Human or Non-Human": 0-1 output indicating if the character appears human (1) or non-human (0). Anthropomorphized characters are not considered humans.
                            - "Physical Features": List of physical features of the character.
                            - "Explanation": Explanation for why the character is classified as human or non-human, and why these physical features are inferred.
                            - "Age": Expected age range of the character (a single number).

                        Ensure that the structure of the JSON output strictly adheres to these standardized labels.
                        """
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 750
    }

    async with aiohttp.ClientSession() as session:
        async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload) as response:
            if response.status != 200:
                print(f"Request failed with status code: {response.status}")
                return None
            
            content = await response.json()
            content_text = content.get('choices', [{}])[0].get('message', {}).get('content', '').strip()

            try:
                content_dict = json.loads(content_text)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                return None

            return content_dict

# Asynchronous function to process scenes from the folder with progress bar
async def process_scenes_output(output_dir, json_output_dir):
    # Create the output directory for JSON files if it doesn't exist
    os.makedirs(json_output_dir, exist_ok=True)

    # List all the files in the output directory that end with '.jpg'
    scenes = sorted([f for f in os.listdir(output_dir) if f.endswith('.jpg')], key=extract_scene_number)

    # Progress bar for scenes
    with tqdm(total=len(scenes), desc="Processing Scenes", unit="scene") as pbar:
        tasks = []
        for i, scene in enumerate(scenes):
            tasks.append(process_single_scene(i, scene, output_dir, json_output_dir, pbar))

        # Run all tasks concurrently
        await asyncio.gather(*tasks)

# Asynchronous function to process a single scene
async def process_single_scene(i, scene, output_dir, json_output_dir, pbar):
    scene_path = os.path.join(output_dir, scene)
    
    # Encode image in base64
    base64_image = await encode_image(scene_path)

    # Perform color analysis
    color_analysis_result = analyze_image_colors(scene_path)

    # Send image to OpenAI for further analysis
    openai_response = await send_image_to_openai(scene_path, base64_image)

    # Check if openai_response is valid (not None or empty)
    if not openai_response:
        print(f"Skipping {scene} due to invalid OpenAI response.")
        pbar.update(1)
        return

    # Combine both results, and include the reference to the image file
    final_output = {
        "Image File": scene,
        "Image Analysis": {
            **color_analysis_result["Color Analysis"],
            **openai_response.get("Image Analysis", {})
        }
    }

    # Save the final output to a JSON file in the specified folder
    output_filename = f'scene_{i + 1}_analysis.json'
    output_path = os.path.join(json_output_dir, output_filename)

    async with aiofiles.open(output_path, 'w') as json_file:
        await json_file.write(json.dumps(final_output, indent=4))

    pbar.update(1)
    print(f"Analysis for {scene} saved as {output_filename}")

# Helper function to extract scene numbers from filenames
def extract_scene_number(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else -1

# Main function to process videos and analyze images with progress bar
def process_videos_in_directory(directory_path, output_base_dir):
    # Get all video files in the directory
    video_files = [f for f in os.listdir(directory_path) if f.endswith(('.mp4', '.avi', '.mkv'))]

    if not video_files:
        print("No video files found in the directory.")
        return

    # Progress bar for videos
    with tqdm(total=len(video_files), desc="Processing Videos", unit="video") as pbar:
        for i, video_file in enumerate(video_files):
            video_path = os.path.join(directory_path, video_file)
            video_name = os.path.splitext(video_file)[0]
            
            # Create a folder named after the video inside the output_base_dir
            video_output_dir = os.path.join(output_base_dir, video_name)
            scenes_output_dir = os.path.join(video_output_dir, 'scenes_output')
            json_output_dir = os.path.join(video_output_dir, 'json_output')

            os.makedirs(scenes_output_dir, exist_ok=True)

            print(f"Processing video {i + 1}/{len(video_files)}: {video_file}")
            
            # Analyze the video and extract scenes
            scenes = analyze_video(video_path)
            extract_frames_imageio(video_path, scenes, scenes_output_dir)

            # Analyze all scenes and save the JSON output
            asyncio.run(process_scenes_output(scenes_output_dir, json_output_dir))

            print(f"Finished processing video: {video_file}")
            pbar.update(1)

# Main script execution
video_directory = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/02_Video_DB'
output_base_directory = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB'

# Process all videos in the directory
process_videos_in_directory(video_directory, output_base_directory)


Processing Videos:   0%|          | 0/2 [00:00<?, ?video/s]ERROR:pyscenedetect:VideoManager is deprecated and will be removed.
INFO:pyscenedetect:Loaded 1 video, framerate: 23.976 FPS, resolution: 1280 x 720
INFO:pyscenedetect:Downscale factor set to 5, effective resolution: 256 x 144
INFO:pyscenedetect:Detecting scenes...


Processing video 1/2: Duck_And_Goose.mkv


INFO:root:Detected 209 scenes:
INFO:root:Scene 1: Start 00:00:00.000 / Frame 0, End 00:00:09.551 / Frame 229
INFO:root:Scene 2: Start 00:00:09.551 / Frame 229, End 00:00:11.887 / Frame 285
INFO:root:Scene 3: Start 00:00:11.887 / Frame 285, End 00:00:26.109 / Frame 626
INFO:root:Scene 4: Start 00:00:26.109 / Frame 626, End 00:00:28.737 / Frame 689
INFO:root:Scene 5: Start 00:00:28.737 / Frame 689, End 00:00:32.115 / Frame 770
INFO:root:Scene 6: Start 00:00:32.115 / Frame 770, End 00:00:33.992 / Frame 815
INFO:root:Scene 7: Start 00:00:33.992 / Frame 815, End 00:00:34.910 / Frame 837
INFO:root:Scene 8: Start 00:00:34.910 / Frame 837, End 00:00:39.498 / Frame 947
INFO:root:Scene 9: Start 00:00:39.498 / Frame 947, End 00:00:45.587 / Frame 1093
INFO:root:Scene 10: Start 00:00:45.587 / Frame 1093, End 00:00:55.430 / Frame 1329
INFO:root:Scene 11: Start 00:00:55.430 / Frame 1329, End 00:00:58.976 / Frame 1414
INFO:root:Scene 12: Start 00:00:58.976 / Frame 1414, End 00:01:00.727 / Frame 1456
I

TimeoutError: 

Request failed with status code: 429
Skipping scene_90_frame_1_13981.jpg due to invalid OpenAI response.
Request failed with status code: 429
Request failed with status code: 429
Request failed with status code: 429
Request failed with status code: 429
Request failed with status code: 429
Skipping scene_199_frame_1_32054.jpg due to invalid OpenAI response.
Skipping scene_145_frame_1_22391.jpg due to invalid OpenAI response.
Skipping scene_195_frame_1_31197.jpg due to invalid OpenAI response.
Skipping scene_130_frame_1_19716.jpg due to invalid OpenAI response.
Skipping scene_126_frame_1_19155.jpg due to invalid OpenAI response.
Request failed with status code: 429
Skipping scene_207_frame_1_33648.jpg due to invalid OpenAI response.
Request failed with status code: 429
Request failed with status code: 429
Skipping scene_185_frame_1_28753.jpg due to invalid OpenAI response.
Skipping scene_105_frame_1_15883.jpg due to invalid OpenAI response.
Request failed with status code: 429
Skipping s

In [6]:
# 20-Ago a las 12:08

In [8]:
import os
import base64
import aiohttp
import asyncio
import json
from PIL import Image
import numpy as np
import colorsys
import aiofiles
import nest_asyncio
from tqdm.asyncio import tqdm

# Apply nest_asyncio to handle the running event loop
nest_asyncio.apply()

# Concurrency limit
semaphore = asyncio.Semaphore(5)

# Function to encode the image asynchronously
async def encode_image(image_path):
    async with aiofiles.open(image_path, "rb") as image_file:
        content = await image_file.read()
        return base64.b64encode(content).decode('utf-8')

# Function to determine the specific primary or special color category
def get_color_category(color):
    r, g, b = [x / 255.0 for x in color]
    h, l, s = colorsys.rgb_to_hls(r, g, b)

    primary_hues = {
        "red": (0.0, 0.1),  
        "yellow": (0.1, 0.18),
        "green": (0.25, 0.4),
        "blue": (0.55, 0.75),
    }

    for color_name, hue_range in primary_hues.items():
        if hue_range[0] <= h <= hue_range[1]:
            return color_name

    if (l >= 0.9 and s <= 0.1):
        return "white"
    if (l <= 0.1 and s <= 0.1):
        return "black"

    return "non-primary"

# Function to analyze image colors and return as a dictionary
def analyze_image_colors(image_path):
    image = Image.open(image_path)
    image = image.convert('RGB')
    data = np.array(image)

    unique_colors, counts = np.unique(data.reshape(-1, data.shape[2]), axis=0, return_counts=True)
    total_pixels = int(counts.sum())

    color_counts = {
        "Red": 0,
        "Yellow": 0,
        "Green": 0,
        "Blue": 0,
        "White": 0,
        "Black": 0,
        "Non-primary": 0
    }

    for color, count in zip(unique_colors, counts):
        category = get_color_category(tuple(color))
        color_counts[category.capitalize()] += int(count)

    color_percentages = {color: (count / total_pixels) * 100 for color, count in color_counts.items()}
    primary_total = color_counts["Red"] + color_counts["Yellow"] + color_counts["Blue"]
    color_dominance = "Primary colors" if primary_total > color_counts["Non-primary"] else "Non-primary colors"

    color_analysis_result = {
        "Color Analysis": {
            "Colors Found": {
                color: {
                    "Pixel Count": count,
                    "Percentage": f"{color_percentages[color]:.2f}%"
                } for color, count in color_counts.items()
            },
            "Dominance": color_dominance
        }
    }
    
    return color_analysis_result

# Asynchronous function to send the image to OpenAI API with retries
# Asynchronous function to send the image to OpenAI API with retries
async def send_image_to_openai(image_path, base64_image, retries=3):
    api_key = os.getenv("OPENAI_API_KEY")
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    # Payload containing your specific prompt
    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": """
                        Analyze the following image and provide a detailed description in the format of JSON only. Ensure the output is strictly in JSON format without any additional text or code block formatting. The JSON should include the following standardized labels:

                        1. **Image Analysis**: The root dictionary containing all analysis data.
                        
                        2. **Suitability**:
                            - "Nudity": Boolean indicating the presence of nudity.
                            - "Obscene Gestures": Boolean indicating the presence of obscene gestures.
                            - "Alcohol": Boolean indicating the presence of alcohol.
                            - "Drugs": Boolean indicating the presence of drugs.
                            - "Addictions": Boolean indicating the presence of addictions.

                        3. **Objects**:
                            - "Total Objects Identified": Integer representing the total number of objects identified.
                            - "Average Features Per Object": Float representing the average number of features per object.
                            - "Objects Details": Dictionary containing details of each object, where each object is labeled as "Object_1", "Object_2", etc., with the following structure:
                                - "Name": The name of the object.
                                - "Color": The color of the object.
                                - "Features": List of features of the object.
                                - "Total Features": Integer representing the number of features for the object.

                        4. **Place**:
                            - "Certainty": String indicating the certainty level of the identified place.
                            - "Fantasy/Adventurous Place": Boolean (0-1) indicating whether the place is classified as a fantasy/adventurous place or not.
                            - "Explanation": Detailed explanation of why the place is classified as fantasy/adventurous or not. Fantasy places are those that do not exist in reality, and adventurous places are defined as those involving clear statements of traveling to space or another country.

                        5. **Characters**:
                            - "Total Characters Identified": Integer representing the total number of characters identified.
                            - "Average Features Per Character": Float representing the average number of features per character.
                            - "Character Details": Dictionary containing details of each character, where each character is labeled as "Character_1", "Character_2", etc., with the following structure:
                                - "Name": The name of the character, if identifiable, or "Unknown" otherwise.
                                - "Human or Non-Human": 0-1 output indicating if the character appears human (1) or non-human (0). Anthropomorphized characters are not considered humans.
                                - "Physical Features": List of physical features of the character.
                                - "Explanation": Explanation for why the character is classified as human or non-human, and why these physical features are inferred.
                                - "Age": Expected age range of the character (a single number).
                            **Note**: If the "character" consists of only a part of a body (such as a hand, leg, or face without enough distinguishing features to identify it as a complete character), do not count it as a "character."

                        Ensure that the structure of the JSON output strictly adheres to these standardized labels.
                        """
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 750
    }

    for attempt in range(retries):
        async with aiohttp.ClientSession() as session:
            async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload) as response:
                if response.status == 429:
                    # Rate limit hit, wait and retry
                    wait_time = 2 ** attempt
                    print(f"Rate limit hit, retrying in {wait_time} seconds...")
                    await asyncio.sleep(wait_time)
                    continue
                elif response.status == 200:
                    content = await response.json()

                    # Safely check if 'choices' exists and if the content is JSON formatted
                    if 'choices' in content:
                        message_content = content['choices'][0].get('message', {}).get('content', '').strip()
                        try:
                            # Try to parse the JSON content
                            content_dict = json.loads(message_content)
                            return content_dict
                        except json.JSONDecodeError as e:
                            print(f"Error decoding JSON from OpenAI response: {e}")
                            return None
                    else:
                        print("Unexpected response format from OpenAI API.")
                        return None
                else:
                    print(f"Request failed with status code: {response.status}")
                    return None
    return None  # Return None after all retries


# Asynchronous function to process scenes from the folder with progress bar
async def process_scenes_output(output_dir, json_output_dir):
    # Create the output directory for JSON files if it doesn't exist
    os.makedirs(json_output_dir, exist_ok=True)

    # List all the files in the output directory that end with '.jpg'
    scenes = sorted([f for f in os.listdir(output_dir) if f.endswith('.jpg')], key=extract_scene_number)

    # Progress bar for scenes
    with tqdm(total=len(scenes), desc="Processing Scenes", unit="scene") as pbar:
        tasks = []
        for i, scene in enumerate(scenes):
            tasks.append(process_single_scene(i, scene, output_dir, json_output_dir, pbar))

        # Run all tasks concurrently
        await asyncio.gather(*tasks)

# Asynchronous function to process a single scene with concurrency control
async def process_single_scene(i, scene, output_dir, json_output_dir, pbar):
    async with semaphore:  # Limit concurrent execution
        scene_path = os.path.join(output_dir, scene)

        # Encode image in base64
        base64_image = await encode_image(scene_path)

        # Perform color analysis
        color_analysis_result = analyze_image_colors(scene_path)

        # Send image to OpenAI for further analysis
        openai_response = await send_image_to_openai(scene_path, base64_image)

        # Check if openai_response is valid (not None or empty)
        if not openai_response:
            print(f"Skipping {scene} due to invalid OpenAI response.")
            pbar.update(1)
            return

        # Combine both results, and include the reference to the image file
        final_output = {
            "Image File": scene,
            "Image Analysis": {
                **color_analysis_result["Color Analysis"],
                **openai_response.get("Image Analysis", {})
            }
        }

        # Save the final output to a JSON file in the specified folder
        output_filename = f'scene_{i + 1}_analysis.json'
        output_path = os.path.join(json_output_dir, output_filename)

        async with aiofiles.open(output_path, 'w') as json_file:
            await json_file.write(json.dumps(final_output, indent=4))

        pbar.update(1)
        print(f"Analysis for {scene} saved as {output_filename}")

# Helper function to extract scene numbers from filenames
def extract_scene_number(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else -1

# Main function to process videos and analyze images with progress bar
def process_videos_in_directory(directory_path, output_base_dir):
    # Get all video files in the directory
    video_files = [f for f in os.listdir(directory_path) if f.endswith(('.mp4', '.avi', '.mkv'))]

    if not video_files:
        print("No video files found in the directory.")
        return

    # Progress bar for videos
    with tqdm(total=len(video_files), desc="Processing Videos", unit="video") as pbar:
        for i, video_file in enumerate(video_files):
            video_path = os.path.join(directory_path, video_file)
            video_name = os.path.splitext(video_file)[0]
            
            # Create a folder named after the video inside the output_base_dir
            video_output_dir = os.path.join(output_base_dir, video_name)
            scenes_output_dir = os.path.join(video_output_dir, 'scenes_output')
            json_output_dir = os.path.join(video_output_dir, 'json_output')

            os.makedirs(scenes_output_dir, exist_ok=True)

            print(f"Processing video {i + 1}/{len(video_files)}: {video_file}")
            
            # Analyze the video and extract scenes
            scenes = analyze_video(video_path)
            extract_frames_imageio(video_path, scenes, scenes_output_dir)

            # Analyze all scenes and save the JSON output
            asyncio.run(process_scenes_output(scenes_output_dir, json_output_dir))

            print(f"Finished processing video: {video_file}")
            pbar.update(1)

# Main script execution
video_directory = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/02_Video_DB'
output_base_directory = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB'

# Process all videos in the directory
process_videos_in_directory(video_directory, output_base_directory)

print("FINISHED PROCESSING ALL VIDEOS.")


Processing Videos:   0%|          | 0/2 [00:00<?, ?video/s]ERROR:pyscenedetect:VideoManager is deprecated and will be removed.
INFO:pyscenedetect:Loaded 1 video, framerate: 23.976 FPS, resolution: 1280 x 720
INFO:pyscenedetect:Downscale factor set to 5, effective resolution: 256 x 144
INFO:pyscenedetect:Detecting scenes...


Processing video 1/2: Duck_And_Goose.mkv


INFO:root:Detected 209 scenes:
INFO:root:Scene 1: Start 00:00:00.000 / Frame 0, End 00:00:09.551 / Frame 229
INFO:root:Scene 2: Start 00:00:09.551 / Frame 229, End 00:00:11.887 / Frame 285
INFO:root:Scene 3: Start 00:00:11.887 / Frame 285, End 00:00:26.109 / Frame 626
INFO:root:Scene 4: Start 00:00:26.109 / Frame 626, End 00:00:28.737 / Frame 689
INFO:root:Scene 5: Start 00:00:28.737 / Frame 689, End 00:00:32.115 / Frame 770
INFO:root:Scene 6: Start 00:00:32.115 / Frame 770, End 00:00:33.992 / Frame 815
INFO:root:Scene 7: Start 00:00:33.992 / Frame 815, End 00:00:34.910 / Frame 837
INFO:root:Scene 8: Start 00:00:34.910 / Frame 837, End 00:00:39.498 / Frame 947
INFO:root:Scene 9: Start 00:00:39.498 / Frame 947, End 00:00:45.587 / Frame 1093
INFO:root:Scene 10: Start 00:00:45.587 / Frame 1093, End 00:00:55.430 / Frame 1329
INFO:root:Scene 11: Start 00:00:55.430 / Frame 1329, End 00:00:58.976 / Frame 1414
INFO:root:Scene 12: Start 00:00:58.976 / Frame 1414, End 00:01:00.727 / Frame 1456
I

Analysis for scene_1_frame_1_0.jpg saved as scene_1_analysis.json


Analysis for scene_2_frame_1_229.jpg saved as scene_2_analysis.json


...

Analysis for scene_157_frame_1_16674.jpg saved as scene_157_analysis.json


Analysis for scene_154_frame_1_16374.jpg saved as scene_154_analysis.json


Analysis for scene_159_frame_1_17000.jpg saved as scene_159_analysis.json


Processing Videos: 100%|██████████| 2/2 [18:14<00:00, 547.18s/video]

Error decoding JSON from OpenAI response: Expecting property name enclosed in double quotes: line 44 column 5 (char 1285)
Skipping scene_158_frame_1_16720.jpg due to invalid OpenAI response.
Finished processing video: Bananas_in_pyjamas.mp4
